In [92]:
#Question1
import requests

In [23]:
import lxml.html as lh

In [24]:
import bs4 as bs

In [26]:
import urllib.request

In [25]:
import numpy as np

In [27]:
import pandas as pd

In [28]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [29]:
def scrape_table_bs4(cname,cols):
    page=urllib.request.urlopen(url).read()
    soup=bs.BeautifulSoup(page,'lxml')
    table=soup.find("table",class_=cname)
    header=[head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data=[[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data=[row for row in data if len(row) == cols]
    raw_df=pd.DataFrame(data,columns=header)
    return raw_df

In [30]:
def scrape_table_lxml(XPATH,cols):
    page=requests.get(url)
    doc=lh.fromstring(page.content)
    table_content=doc.xpath(XPATH)
    for table in table_content:
        headers=[th.text_content().strip() for th in table.xpath('//th')]
        headers=headers[0:3]
        data=[[td.text_content().strip() for td in tr.xpath('td')] 
                   for tr in table.xpath('//tbody/tr')]
        data=[row for row in data if len(row) == cols]
        raw_df=pd.DataFrame(data,columns=headers)
        return raw_df

In [31]:
raw_TorontoPostalCodes = scrape_table_bs4("wikitable",3)

In [32]:
TorontoPostalCodes=raw_TorontoPostalCodes[~raw_TorontoPostalCodes['Borough'].isin(['Not assigned'])]

In [33]:
TorontoPostalCodes=TorontoPostalCodes.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

In [34]:
TorontoPostalCodes.loc[TorontoPostalCodes['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = TorontoPostalCodes['Borough']

In [35]:
check_unassigned_post_state_sample = TorontoPostalCodes.loc[TorontoPostalCodes['Borough'] == 'Queen\'s Park']

In [36]:
TorontoPostalCodes = TorontoPostalCodes.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [37]:
TorontoPostalCodes

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [48]:
#Question 2
TorontoPostalCodes.to_csv('Toronto.TASK_1_df.csv',index=False)

In [49]:
task1_data_csv="Toronto.TASK_1_df.csv"

In [50]:
TorontoPostalCodes=pd.read_csv(task1_data_csv).set_index("Postcode")

In [51]:
TorontoPostalCodes.rename_axis("Postal Code", axis='index', inplace=True)

In [52]:
toronto_geocsv='https://cocl.us/Geospatial_data'

In [53]:
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv

In [54]:
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")

In [55]:
toronto_neighborhoods = TorontoPostalCodes.join(geocsv_data)

In [56]:
toronto_neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [57]:
#Question 3

In [58]:
toronto_neighborhoods.to_csv('Toronto.TASK_II_df.csv',index=False)

In [59]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [60]:
pd.set_option('display.max_columns', None)

In [61]:
pd.set_option('display.max_rows', None)

In [62]:
import json

In [63]:
from geopy.geocoders import Nominatim 

In [64]:
GeoLocator = Nominatim(user_agent='My-IBMNotebook')

In [65]:
import requests 

In [66]:
from pandas.io.json import json_normalize

In [67]:
import matplotlib.cm as cm

In [68]:
import matplotlib.colors as colors

In [69]:
from sklearn.cluster import KMeans

In [71]:
!pip install folium

     |████████████████████████████████| 92kB 9.0MB/s eta 0:00:011


In [72]:
import folium

In [73]:
toronto_task1_csv = "Toronto.TASK_1_df.csv"

In [74]:
toronto_task2_csv = "Toronto.TASK_II_df.csv"

In [75]:
toronto_neighborhoods = pd.read_csv(toronto_task2_csv)

In [76]:
toronto_neighborhoods.shape

(103, 4)

In [77]:
toronto_neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [78]:
address = 'Toronto'

In [79]:
geolocator = Nominatim()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':


In [80]:
location = geolocator.geocode(address)

In [81]:
latitude = location.latitude

In [82]:
longitude = location.longitude

In [83]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

In [84]:
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)

In [85]:
map_toronto

In [88]:
toronto_data = toronto_neighborhoods[toronto_neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)

In [90]:
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(map_toronto)  

In [91]:
map_toronto